In [2]:
import pandas
import numpy as np

In [1]:
from sklearn.model_selection import GridSearchCV, train_test_split,cross_val_score,StratifiedKFold,KFold
from sklearn.metrics import confusion_matrix,accuracy_score,silhouette_score,calinski_harabasz_score
from sklearn.feature_selection import SelectKBest,f_classif,SelectFdr
from sklearn import svm
from sklearn import preprocessing
from matplotlib import pyplot as plt
from sklearn.preprocessing import normalize,RobustScaler
from sklearn.cluster import KMeans
from lifelines import CoxPHFitter
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import backend
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import glorot_uniform,RandomUniform,Constant

In [3]:
# read data
methy = pandas.read_csv("D:/1_Jie_OIO/pancreatic_cancer/input data/methylation2.csv")
mrna = pandas.read_csv("D:/1_Jie_OIO/pancreatic_cancer/input data/mrna.csv")
mirna = pandas.read_csv("D:/1_Jie_OIO/pancreatic_cancer/input data/mirna.csv")
clinical_new = pandas.read_csv("D:/1_Jie_OIO/pancreatic_cancer/clinical_data3.csv")

In [4]:
## clean up the training sets

# drop redundant columns 
methy = methy.drop(['Unnamed: 0'], axis=1)
mrna = mrna.drop(['Unnamed: 0'], axis=1)
mirna = mirna.drop(['Unnamed: 0'], axis=1)

clinical_new = clinical_new.drop(['Unnamed: 0'], axis = 1)
clinical_new = clinical_new[['bcr_patient_barcode','vital_status','survival','cause_of_death']]

# reset index
mrna = mrna.set_index(['Group.1'])
mirna = mirna.set_index(['GeneSymbol'])

clinical_new.reset_index(inplace=True)
# transpose
methy = methy.transpose()
mrna = mrna.transpose()
mirna = mirna.transpose()

# vital status has to be 0/1 not 1/2
clinical_new[["vital_status"]] = clinical_new[["vital_status"]] -1

In [5]:
# data log2 transformation
mrna = np.log2(mrna+1)
mirna = np.log2(mirna+1)

In [6]:
# concatenate the multi-omics data
data_all = pandas.concat([methy,mrna,mirna],axis = 1)
data_all2 = data_all.loc[clinical_new['bcr_patient_barcode'],:]
data_all2

,0,1,2,3,4,5,6,7,8,9,...,hsa-mir-937,hsa-mir-939,hsa-mir-940,hsa-mir-942,hsa-mir-944,hsa-mir-95,hsa-mir-96,hsa-mir-98,hsa-mir-99a,hsa-mir-99b
TCGA.2J.AAB6,0.392125,0.443796,0.470523,0.587413,0.730783,0.481929,0.401216,0.0816138,0.631308,0.789943,...,1.659146,0.310120,1.764731,0.970296,0.565228,2.806379,4.812427,5.570562,8.939383,14.976723
TCGA.2J.AAB8,0.503784,0.657865,0.671,0.747297,0.745571,0.465296,0.555999,0.0653063,0.688562,0.773821,...,0.835161,1.121734,1.500529,1.565665,0.606636,2.512059,3.122654,5.361682,8.882110,13.226844
TCGA.2J.AAB9,0.40715,0.55115,0.607378,0.714571,0.774303,0.478354,0.558286,0.0969258,0.702072,0.846118,...,0.282327,0.282327,0.898881,1.448498,1.199706,2.290066,3.169561,5.349524,9.336851,14.191277
TCGA.2J.AABA,0.537019,0.692183,0.677414,0.646668,0.744123,0.485663,0.528758,0.0622114,0.670515,0.819271,...,0.000000,0.253555,0.469137,1.724951,0.656654,0.656654,1.724951,4.576809,10.267290,13.527318
TCGA.2J.AABE,0.485248,0.632153,0.649158,0.756352,0.739516,0.485606,0.497274,0.0869353,0.647609,0.786354,...,1.158019,0.611445,0.434945,1.553472,0.768687,3.266121,3.463099,5.252521,9.104412,13.860464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA.XD.AAUL,0.430066,0.616884,0.601854,0.727783,0.716007,0.465204,0.560235,0.0683619,0.656267,0.788165,...,1.521978,0.501787,1.168577,2.367276,1.168577,1.521978,3.587736,5.772403,9.206623,13.619919
TCGA.XN.A8T3,0.389799,0.508017,0.622184,0.761291,0.710781,0.480178,0.544202,0.0650754,0.669153,0.834831,...,0.761530,0.231214,1.148449,2.824518,2.751920,4.279853,4.506384,5.888266,9.208170,14.478188
TCGA.YB.A89D,0.505117,0.648158,0.712331,0.79111,0.744692,0.486454,0.604904,0.0652477,0.662704,0.822138,...,0.143124,0.143124,0.701117,1.765008,0.955049,2.427698,3.676404,5.942291,10.096909,13.867643
TCGA.YY.A8LH,0.568815,0.737544,0.500254,0.773945,0.655661,0.541389,0.58206,0.0848873,0.619335,0.862767,...,1.051422,0.689655,1.340416,1.721752,0.205673,2.833695,5.227565,6.105980,9.825959,14.498457


# train autoencoder for 146 samples

# functions

In [7]:
def normalize_ae(X_train):
    # normalize each data type set

    # split dataset into multi omics data
    methy_train = X_train.iloc[:,0:20980]
    mrna_train = X_train.iloc[:,20980:38168]
    mirna_train = X_train.iloc[:,38168:38597]

    # l2 normalization,sample norm
    methy_train = normalize(methy_train, norm='l2',axis = 1)
    mrna_train = normalize(mrna_train, norm='l2',axis = 1)
    mirna_train = normalize(mirna_train, norm='l2',axis = 1)

    data = pandas.concat([pandas.DataFrame(methy_train),pandas.DataFrame(mrna_train),pandas.DataFrame(mirna_train)],axis = 1)
    
    return data

In [8]:
def coxph_feature_selection(data,clinical_data):
    p_value = []
    cph = CoxPHFitter()
    for j in range(0,len(data.columns)):
        data_label = pandas.concat([data[j],clinical_data[["survival","vital_status"]]],axis = 1)
        cph.fit(data_label, duration_col="survival", event_col="vital_status")
        # get p value
        if cph.summary.iloc[0,4] <0.05:
            p_value.append(j)
            print(j)
    data_new = data[p_value]
    print(len(p_value))    
    return data_new         

In [9]:
def kmeans_function(data,cluster):
    for n in range(2,6):
        kmeans = KMeans(n_clusters=n, n_init =10).fit(data)
        labels = kmeans.labels_
        print(silhouette_score(data, labels))
        print(calinski_harabasz_score(data, labels))
    kmeans = KMeans(n_clusters=cluster, n_init=10).fit(data)
    labels = kmeans.labels_
    return labels

In [126]:
seed_num = 1000

def paad_model(activation = "tanh",hidden_layers = 500, bottleneck = 100,l2 = 0.001,l1=0.001):
    
    model = Sequential()
    
    model.add(Dense(hidden_layers,activation= activation,input_shape=(38597,),kernel_regularizer=regularizers.l2(l2),
                    activity_regularizer=regularizers.l1(l1),kernel_initializer=glorot_uniform(seed = seed_num)))#             
    
    model.add(Dropout(0.5))
    
    model.add(Dense(bottleneck, activation=activation,kernel_initializer=glorot_uniform(seed = seed_num)))
    #,random_uniform,Constant(value=0.005),glorot_uniform
                    
    model.add(Dropout(0.5))
    
    model.add(Dense(hidden_layers, activation=activation,kernel_initializer=glorot_uniform(seed = seed_num))) 
    
    model.add(Dropout(0.5))
    
    model.add(Dense(38597, activation=activation,kernel_initializer=glorot_uniform(seed = seed_num)))
    
    model.compile(loss='mean_squared_logarithmic_error',optimizer='sgd') #,mean_squared_error
    
    return model

# run

In [129]:
df = normalize_ae(data_all2)

model = paad_model('tanh',500,200,0.001,0.0001)
autoencoder_train = model.fit(x=df, y=df, epochs=10, batch_size=1) #,validation_split=0.2

# get bottleneck layer
layers = backend.function([model.layers[0].input],[model.layers[2].output])
feature_new = pandas.DataFrame(layers([df])[0])

# calculate p value
df_new = coxph_feature_selection(feature_new,clinical_new)
label_all = kmeans_function(df_new,2)
label_all

Train on 116 samples, validate on 30 samples
Epoch 1/10
116/116 [==============================] - 18s 159ms/sample - loss: 0.9854 - val_loss: 0.9830
Epoch 2/10
116/116 [==============================] - 15s 132ms/sample - loss: 0.9808 - val_loss: 0.9785
Epoch 3/10
116/116 [==============================] - 15s 132ms/sample - loss: 0.9762 - val_loss: 0.9739
Epoch 4/10
116/116 [==============================] - 15s 132ms/sample - loss: 0.9717 - val_loss: 0.9694
Epoch 5/10
116/116 [==============================] - 15s 132ms/sample - loss: 0.9672 - val_loss: 0.9649
Epoch 6/10
116/116 [==============================] - 15s 132ms/sample - loss: 0.9627 - val_loss: 0.9604
Epoch 7/10
116/116 [==============================] - 15s 132ms/sample - loss: 0.9582 - val_loss: 0.9559
Epoch 8/10
116/116 [==============================] - 15s 132ms/sample - loss: 0.9537 - val_loss: 0.9515
Epoch 9/10
116/116 [==============================] - 15s 132ms/sample - loss: 0.9493 - val_loss: 0.9471
Epoch 10/1

C:\Users\Jie\Anaconda3.7\lib\site-packages\lifelines\utils\__init__.py:1091: ConvergenceWarning: Column(s) [0] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
C:\Users\Jie\Anaconda3.7\lib\site-packages\lifelines\utils\__init__.py:1091: ConvergenceWarning: Column(s) [1] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
C:\Users\Jie\Anaconda3.7\lib\site-packages\lifelines\utils\__init__.py:1091: ConvergenceWarning: Column(s) [2] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fittin

9


C:\Users\Jie\Anaconda3.7\lib\site-packages\lifelines\utils\__init__.py:1091: ConvergenceWarning: Column(s) [10] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
C:\Users\Jie\Anaconda3.7\lib\site-packages\lifelines\utils\__init__.py:1091: ConvergenceWarning: Column(s) [11] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
C:\Users\Jie\Anaconda3.7\lib\site-packages\lifelines\utils\__init__.py:1091: ConvergenceWarning: Column(s) [12] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fit

17


...

C:\Users\Jie\Anaconda3.7\lib\site-packages\lifelines\utils\__init__.py:1091: ConvergenceWarning: Column(s) [167] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
C:\Users\Jie\Anaconda3.7\lib\site-packages\lifelines\utils\__init__.py:1091: ConvergenceWarning: Column(s) [168] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
C:\Users\Jie\Anaconda3.7\lib\site-packages\lifelines\utils\__init__.py:1091: ConvergenceWarning: Column(s) [169] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before 

182


C:\Users\Jie\Anaconda3.7\lib\site-packages\lifelines\utils\__init__.py:1091: ConvergenceWarning: Column(s) [183] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
C:\Users\Jie\Anaconda3.7\lib\site-packages\lifelines\utils\__init__.py:1091: ConvergenceWarning: Column(s) [184] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
C:\Users\Jie\Anaconda3.7\lib\site-packages\lifelines\utils\__init__.py:1091: ConvergenceWarning: Column(s) [185] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before 

195
16
0.15743917
28.006081046228633
0.11881311
21.896293419995143
0.09692291
17.973581267168203
0.08731041
16.08265242947292


array([0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1])

In [ ]:
#1e-6
label_all = [0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1,
             1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1,         
             0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1,
             1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1,
             0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
             1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0,
             0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1]

In [ ]:
label_all = [0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1,
             1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
             1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
             1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1,
             1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
             1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
             0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1]